In [1]:
# Imports
import sys
sys.path.append("..")
from os import listdir
from os.path import isfile, join
import h5py
import os
import torch
import torch.nn as nn 
import torch.optim as optim
import torch.functional as F
import torch.backends.cudnn as cudnn
from torch.autograd import Variable
from torch.utils.data import Dataset
# Local Imports
from src.models.CNN import CNNModel
from src.data.voxelize import voxelize
from src.data.utils import read_voxel_data

In [2]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
       
# Paths
LABELED_DATA = os.path.abspath("../data/labeled/")
VOXELIZED_DATA = os.path.abspath("../data/voxelized/amazon.h5py")
VOXEL_DIR = os.path.abspath("../data/tiles/")
# Model Variables 
BATCH_SIZE = 1
NUM_CLASSES = 2
NUM_EPOCHS = 1
LEARNING_RATE = 0.001

In [3]:
# Custom dataset
class VoxelDataset(Dataset):
    def __init__(self, voxel_dir, transform=None, target_transform=None):
        self.voxel_dir = voxel_dir
        self.transform = transform
        self.target_transform = target_transform
        self.voxel_files = [f for f in listdir(voxel_dir) if isfile(join(voxel_dir, f))]

    def __len__(self):
        return len(self.voxel_files)

    def __getitem__(self, idx):
        voxel_path = os.path.join(self.voxel_dir, self.voxel_files[idx])
        #voxels, labels = read_voxel_data(voxel_path)
        #voxels = torch.from_numpy(voxels).float()
        #labels = torch.from_numpy(labels).long()
        with h5py.File(voxel_path, "r") as hf:
            arr = hf['default'][:]
        voxels = arr[:,:,:,0:7]
        labels = arr[:,:,:,-1]
        if self.transform:
            voxels = self.transform(voxels)
        if self.target_transform:
            labels = self.target_transform(labels)
        return voxels, labels


In [4]:
# Create dataset and dataloader
train = VoxelDataset(VOXEL_DIR)
train_loader = torch.utils.data.DataLoader(train, batch_size = BATCH_SIZE, shuffle = True)

In [5]:
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, filter_size, padding=1, n_features=7):
        super(ConvBlock, self).__init__()
        self.conv = nn.Sequential(
            nn.BatchNorm3d(in_channels),
            nn.ReLU(),
            nn.Conv3d(in_channels, out_channels, filter_size, padding=padding),
            nn.BatchNorm3d(out_channels),
            nn.ReLU(),
            nn.Dropout3d(p=0.2),
            nn.Conv3d(out_channels, out_channels, filter_size, padding=padding),
                     )
    def forward(self, x):
        o = self.conv(x)
        return o

    
class MinimalSegmenter(nn.Module):
    def __init__(self, in_channels, n_classes):
        super(MinimalSegmenter, self).__init__()
        self.drop = nn.Dropout3d(p=0.2)
        self.mp = nn.MaxPool3d((2,2,2))
        self.e1 = ConvBlock(in_channels, 32, 3)
        self.e2 = ConvBlock(32, 64, 3)
        self.e3 = ConvBlock(64,128, 3)
        self.d1 = nn.ConvTranspose3d(128, 64, 3, stride=2, padding=1)
        self.dc1 = ConvBlock(128, 64, 3)
        self.d2 = nn.ConvTranspose3d(64, 32, 3, stride=2, padding=1)
        self.dc2 = ConvBlock(64, 32,3)
        self.out = nn.Conv3d(32,n_classes,1)
    def forward(self, x):
        o = self.e1(x)
        s1 = o
        print("s1: ",s1.size())
        o = self.mp(o)
        o = self.drop(o)
        o = self.e2(o)
        s2 = o
        print("s2: ",s2.size())
        o = self.mp(o)
        o = self.drop(o)
        o = self.e3(o)
        
        o = self.d1(o)
        print("d1: ", o.size())
        o = torch.cat((o, s2))
        o = self.drop(o)
        o = self.dc1(o)
        
        o = self.d2(o)
        print("d2: ",o.size())
        o = torch.cat((o, s2))
        o = self.drop(o)
        o = self.dc2(o)
        
        o = self.out(o)
        o = F.log_softmax(o, dim=3)
        return o
    
# Contains bugs, WIP.
class ForestSegmenter(nn.Module):
    def __init__(self, in_channels, n_classes):
        super(ForestSegmenter, self).__init__()
        # dropout layer
        self.drop = nn.Dropout3d(p=0.2)
        
        # pooling layers
        self.mp = nn.MaxPool3d((2,2,2))

        # encoder part
        self.e1 = ConvBlock(in_channels, 32, 3)
        self.e2 = ConvBlock(32,64,3)
        self.e3 = ConvBlock(64,128,3)
        self.e4 = ConvBlock(128,256,3)
        self.e5 = ConvBlock(256,512,3)
        
        # decoder part
        self.d1 = nn.ConvTranspose3d(512,256,3, stride=2,  padding=1)
        self.dc1 = ConvBlock(512, 256, 3)
        self.d2 = nn.ConvTranspose3d(256,128,3, stride=2, padding=1)
        self.dc2 = ConvBlock(256,128,3)
        self.d3 = nn.ConvTranspose3d(128,64,3, stride=2,  padding=1)
        self.dc3 = ConvBlock(128,64,3)
        self.d4 = nn.ConvTranspose3d(64,32,3, stride=2,  padding=1)
        self.dc4 = ConvBlock(64,32,3)
        
        # output
        self.out = nn.Conv3d(32,n_classes,1)
    def forward(self, x):
        o = self.e1(x)
        s1 = o
        o = self.mp(o)
        o = self.drop(o)
        print(o.size())

        o = self.e2(o)
        s2 = o
        o = self.mp(o)
        o = self.drop(o)
        print(o.size())
        
        o = self.e3(o)
        s3 = o
        o = self.mp(o)
        o = self.drop(o)
        print(o.size())
        
        o = self.e4(o)
        s4 = o
        o = self.mp(o)
        o = self.drop(o)
        print(o.size())
        
        o = self.e5(o)
        print(o.size())

  
        # decode
        print("decoding")
        o = self.d1(o)
        print(o.size())

        o = torch.cat((o, s4))
        o = self.drop(o)
        o = self.dc1(o)
        print(o.size())
        
        o = self.d2(o)
        print(o.size())

        o = torch.cat((o, s3))
        o = self.drop(o)
        o = self.dc2(o)
        print(o.size())
        
        o = self.d3(o)
        o = torch.cat((o, s2))
        o = self.drop(o)
        o = self.dc3(o)
        
        o = self.d4(o)
        o = torch.cat((o, s1))
        o = self.drop(o)
        o = self.dc3(o)
        
        o = self.out(o)
        o = F.log_softmax(o, dim=3)
        return o
        
    
    

In [6]:
# Model
 
model = MinimalSegmenter(7,2).to(device)
loss_func = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE)


In [7]:
# Training 
for epoch in range(NUM_EPOCHS):
    for i, (x, y) in enumerate(train_loader):
        x = x.view(x.size(0), x.size(4), x.size(1), x.size(2), x.size(3)).to(device)
        y = y.view(y.size(0), y.size(1), y.size(2), y.size(3)).to(device)
        optimizer.zero_grad()
        y_ = model(x)
        loss = loss_func(y_, y)
        loss.backward()
        optimizer.step()
        del x,y,y_
        torch.cuda.empty_cache()


s1:  torch.Size([1, 32, 100, 100, 370])
s2:  torch.Size([1, 64, 50, 50, 185])
d1:  torch.Size([1, 64, 49, 49, 183])


RuntimeError: Sizes of tensors must match except in dimension 0. Expected size 49 but got size 50 for tensor number 1 in the list.